In [1]:
%cd ../../../..
%pwd

/sc/arion/projects/pandeg01a/susmaa01/lei


/sc/arion/projects/pandeg01a/susmaa01/lei/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/sc/arion/projects/pandeg01a/susmaa01/lei'

In [2]:
import pickle as pkl
import pandas as pd
import numpy as np

In [14]:
with open("/sc/arion/projects/pandeg01a/susmaa01/lei/longitudinal_tadpole/tadpole_data/tadpole_data_as_dfs/tadpole_labels_time_imptn_norm_thrshld30_dfs.pickle", "rb") as file:
    labels = pkl.load(file=file)
with open("/sc/arion/projects/pandeg01a/susmaa01/lei/longitudinal_tadpole_v2/bps/results/v1/no_sampling/0.pkl", "rb") as file:
    ei = pkl.load(file=file)

In [31]:
id_vis = np.concatenate([np.stack([np.arange(749), np.zeros(749) + t]).T for t in range(4)])
id_vis.shape

(2996, 2)

In [32]:
y = np.concatenate([labels.values[:,t] for t in range(4)])
y.shape

(2996,)

In [33]:
ei.groups.shape

(2996,)

In [95]:
ensemble_test_id = [id_vis[test_idx] for _, test_idx in ei.cv_outer.split(id_vis, y, groups=ei.groups)]
ensemble_train_id = []
for train_idx, _ in ei.cv_outer.split(id_vis, y, groups=ei.groups):
    id_o, y_o, groups_o = id_vis[train_idx], y[train_idx], ei.groups[train_idx]
    ensemble_train_id_fold = np.concatenate([id_o[test_idx] for _, test_idx in ei.cv_inner.split(id_o, y_o, groups=groups_o)], axis=0)
    ensemble_train_id.append(ensemble_train_id_fold)

In [96]:
def ids_to_idx(id_list):
    for i, ids in enumerate(id_list):
        if 0 in np.unique(ids[:,1]):
            ids[:,1] = ids[:,1] + 1
            id_list[i] = [tuple(x) for x in ids.astype(int)]
    return id_list

ensemble_test_id = ids_to_idx(ensemble_test_id)
ensemble_train_id = ids_to_idx(ensemble_train_id)

In [97]:
ensemble_test_id[0]

[(1, 1),
 (12, 1),
 (16, 1),
 (21, 1),
 (22, 1),
 (31, 1),
 (35, 1),
 (38, 1),
 (49, 1),
 (52, 1),
 (55, 1),
 (59, 1),
 (70, 1),
 (71, 1),
 (78, 1),
 (84, 1),
 (89, 1),
 (92, 1),
 (95, 1),
 (101, 1),
 (105, 1),
 (107, 1),
 (112, 1),
 (118, 1),
 (124, 1),
 (127, 1),
 (141, 1),
 (145, 1),
 (147, 1),
 (149, 1),
 (156, 1),
 (166, 1),
 (167, 1),
 (170, 1),
 (178, 1),
 (185, 1),
 (186, 1),
 (189, 1),
 (191, 1),
 (196, 1),
 (202, 1),
 (215, 1),
 (217, 1),
 (220, 1),
 (226, 1),
 (234, 1),
 (237, 1),
 (241, 1),
 (245, 1),
 (248, 1),
 (256, 1),
 (264, 1),
 (269, 1),
 (271, 1),
 (276, 1),
 (282, 1),
 (291, 1),
 (292, 1),
 (301, 1),
 (303, 1),
 (308, 1),
 (313, 1),
 (315, 1),
 (322, 1),
 (325, 1),
 (326, 1),
 (332, 1),
 (335, 1),
 (342, 1),
 (346, 1),
 (359, 1),
 (362, 1),
 (365, 1),
 (368, 1),
 (371, 1),
 (380, 1),
 (381, 1),
 (394, 1),
 (397, 1),
 (398, 1),
 (400, 1),
 (406, 1),
 (410, 1),
 (416, 1),
 (425, 1),
 (429, 1),
 (435, 1),
 (440, 1),
 (447, 1),
 (450, 1),
 (454, 1),
 (459, 1),
 (464, 1

In [99]:
for i in range(ei.k_outer):
    ei.ensemble_training_data[i]['labels'] = [labels.values[x] for x in ensemble_train_id[i]]
    ei.ensemble_test_data[i]['labels'] = [labels.values[x] for x in ensemble_test_id[i]]